In [36]:
import pandas as pd 

df = pd.read_csv('megapc_products.csv')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_name       47 non-null     object
 1   brand              47 non-null     object
 2   series             47 non-null     object
 3   model              47 non-null     object
 4   screen_size        47 non-null     object
 5   screen_resolution  47 non-null     object
 6   screen_type        47 non-null     object
 7   cpu                47 non-null     object
 8   gpu                47 non-null     object
 9   ram                47 non-null     object
 10  storage            47 non-null     object
 11  os                 47 non-null     object
 12  price              47 non-null     object
 13  image_url          47 non-null     object
 14  product_url        47 non-null     object
dtypes: object(15)
memory usage: 5.6+ KB


In [37]:


df['cpu'].unique()

array(['Intel core I5-13420H', 'Intel core I5-12450HX',
       'Intel Ultra core 5 210H', 'Intel core i5- 13450HX',
       'Intel core i7-13620H', 'Intel core I5-13450HX',
       'Intel Ultra core 7 240H', 'Intel core I5-12500H'], dtype=object)

In [38]:


df['gpu'].unique()


array(['RTX 2050', 'RTX 3050', 'RTX 4050', 'RTX 4060', 'RTX 5050'],
      dtype=object)

The creation of a sales dummy dataset

In [39]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Convert price column from string to numeric
df['price'] = (
    df['price']
    .str.replace(' DT', '', regex=False)        # remove " DT"
    .str.replace('\u202f', '', regex=False)     # remove narrow no-break spaces
    .str.replace(' ', '', regex=False)          # remove regular spaces
    .astype(float)
)

# Add TVA percentage (random between 1-10%)
np.random.seed(42)  # for reproducibility
df['tva_percentage'] = np.random.uniform(1, 10, size=len(df))

# Add stock quantity (random between 0-100)
df['stock_quantity'] = np.random.randint(0, 101, size=len(df))

# Add buying price (20-40% less than selling price using normal distribution)
# Mean discount: 30%, std: 5% (to keep it within 20-40% range mostly)
discount_percentage = np.clip(np.random.normal(30, 5, size=len(df)), 20, 40)
df['buying_price'] = df['price'] * (1 - discount_percentage / 100)

# Calculate price with TVA
df['price_with_tva'] = df['price'] * (1 + df['tva_percentage'] / 100)

# Calculate profit margin
df['profit_margin'] = ((df['price'] - df['buying_price']) / df['buying_price'] * 100)

print("Updated DataFrame:")
df[['product_name', 'buying_price', 'price', 'profit_margin', 'tva_percentage', 'stock_quantity', 'price_with_tva']].head()

Updated DataFrame:


,product_name,buying_price,price,profit_margin,tva_percentage,stock_quantity,price_with_tva
0,MSI THIN 15 B13UCX FHD | INTEL CORE I5-13420H ...,1403.110582,2075.0,47.885707,4.370861,5,2165.695367
1,MSI THIN 15 B13UCX FHD | INTEL CORE I5-13420H ...,1351.525258,2129.0,57.525728,9.556429,53,2332.456368
2,Pc portable HP Victus 15-fa1007nk | i5-13420H ...,1481.564976,2149.0,45.049325,7.587945,3,2312.064948
3,MSI THIN 15 B13UCX FHD | INTEL CORE I5-13420H ...,1453.550508,2169.0,49.220821,6.387926,53,2307.554123
4,Pc portable HP Victus 15-fa1007nk | i5-13420H ...,1634.872786,2195.0,34.261211,2.404168,92,2247.771482


In [40]:
# Create dummy sales dataset with profit calculations
def generate_sales_data(df, start_date='2024-01-01', end_date='2024-10-19'):
    """Generate realistic sales data for products with profit tracking"""
    
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    date_range = pd.date_range(start, end, freq='D')
    
    sales_data = []
    
    for date in date_range:
        # Random number of sales per day (0-15 transactions)
        num_sales = np.random.poisson(5)
        
        for _ in range(num_sales):
            # Select random product
            product_idx = np.random.choice(df.index)
            product = df.loc[product_idx]
            
            # Random quantity (1-5 units per transaction)
            quantity = np.random.randint(1, 6)
            
            # Add some time variation within the day
            sale_time = date + timedelta(
                hours=np.random.randint(8, 22),
                minutes=np.random.randint(0, 60)
            )
            
            # Calculate profit
            unit_profit = product['price'] - product['buying_price']
            total_profit = unit_profit * quantity
            
            sales_data.append({
                'sale_date': sale_time,
                'product_name': product['product_name'],
                'brand': product['brand'],
                'series': product['series'],
                'cpu': product['cpu'],
                'gpu': product['gpu'],
                'ram': product['ram'],
                'quantity': quantity,
                'buying_price': product['buying_price'],
                'unit_price': product['price'],
                'unit_profit': unit_profit,
                'tva_percentage': product['tva_percentage'],
                'unit_price_with_tva': product['price_with_tva'],
                'total_amount': product['price_with_tva'] * quantity,
                'total_cost': product['buying_price'] * quantity,
                'total_profit': total_profit,
                'profit_margin': product['profit_margin']
            })
    
    sales_df = pd.DataFrame(sales_data)
    
    # Add temporal features for analysis
    sales_df['date'] = sales_df['sale_date'].dt.date
    sales_df['year'] = sales_df['sale_date'].dt.year
    sales_df['month'] = sales_df['sale_date'].dt.month
    sales_df['week'] = sales_df['sale_date'].dt.isocalendar().week
    sales_df['day_of_week'] = sales_df['sale_date'].dt.day_name()
    sales_df['hour'] = sales_df['sale_date'].dt.hour
    
    return sales_df

# Generate sales data
sales_df = generate_sales_data(df)

print(f"\nTotal sales records: {len(sales_df)}")
print(f"\nDate range: {sales_df['sale_date'].min()} to {sales_df['sale_date'].max()}")
print(f"\nTotal revenue: {sales_df['total_amount'].sum():,.2f} DT")
print(f"Total cost: {sales_df['total_cost'].sum():,.2f} DT")
print(f"Total profit: {sales_df['total_profit'].sum():,.2f} DT")
print(f"Average profit margin: {sales_df['profit_margin'].mean():.2f}%")

sales_df.head(10)


Total sales records: 1418

Date range: 2024-01-01 10:48:00 to 2024-10-19 17:58:00

Total revenue: 11,936,543.00 DT
Total cost: 7,955,959.71 DT
Total profit: 3,412,694.29 DT
Average profit margin: 43.81%


,sale_date,product_name,brand,series,cpu,gpu,ram,quantity,buying_price,unit_price,...,total_amount,total_cost,total_profit,profit_margin,date,year,month,week,day_of_week,hour
0,2024-01-01 11:48:00,"MSI Thin 15 B13UDX-3257XFR | FHD 15.6"" IPS | I...",MSI,Thin,Intel core i7-13620H,RTX 3050,8 GB,1,2102.761470,2799.0,...,2919.280212,2102.761470,696.238530,33.110676,2024-01-01,2024,1,1,Monday,11
1,2024-01-01 10:48:00,"Lenovo LOQ 15IAX9 15.6"" FHD IPS | i5-12450HX |...",Lenovo,LOQ,Intel core I5-12450HX,RTX 2050,24 GB,2,1831.140551,2335.0,...,5124.353111,3662.281103,1007.718897,27.516154,2024-01-01,2024,1,1,Monday,10
2,2024-01-01 15:16:00,MSI CYBORG 15 A13VE FHD | i5-13420H | RTX 4050...,MSI,CYBORG,Intel core I5-13420H,RTX 4050,16 GB,1,2057.947072,3157.0,...,3275.119910,2057.947072,1099.052928,53.405306,2024-01-01,2024,1,1,Monday,15
3,2024-01-01 19:53:00,MSI THIN 15 B13UCX FHD | INTEL CORE I5-13420H ...,MSI,THIN,Intel core I5-13420H,RTX 2050,16 GB,2,1351.525258,2129.0,...,4664.912737,2703.050516,1554.949484,57.525728,2024-01-01,2024,1,1,Monday,19
4,2024-01-02 13:31:00,"ASUS V16 V3607VU-RP246 | 16"" FHD | Intel 7 240...",ASUS,V16,Intel Ultra core 7 240H,RTX 4050,16 GB,2,2433.734986,3179.0,...,6941.911320,4867.469971,1490.530029,30.622275,2024-01-02,2024,1,1,Tuesday,13
5,2024-01-02 15:58:00,MSI THIN 15 B13UCX FHD | INTEL CORE I5-13420H ...,MSI,THIN,Intel core I5-13420H,RTX 2050,24 GB,3,1453.550508,2169.0,...,6922.662368,4360.651524,2146.348476,49.220821,2024-01-02,2024,1,1,Tuesday,15
6,2024-01-02 11:33:00,"DELL G15 5530 | 15.6"" | i5- 13450HX | RTX 3050...",DELL,G15,Intel core i5- 13450HX,RTX 3050,16 GB,5,1817.164346,2459.0,...,12754.609254,9085.821731,3209.178269,35.320727,2024-01-02,2024,1,1,Tuesday,11
7,2024-01-03 11:59:00,MSI CYBORG 15 A13VE FHD | i5-13420H | RTX 4050...,MSI,CYBORG,Intel core I5-13420H,RTX 4050,8 GB,3,1831.188912,2959.0,...,9439.067772,5493.566736,3383.433264,61.589008,2024-01-03,2024,1,1,Wednesday,11
8,2024-01-03 17:02:00,ASUS V16 V3607VU-RP209 | Intel 5 210H | RTX 40...,ASUS,V16,Intel Ultra core 5 210H,RTX 4050,8 GB,3,2062.471447,2899.0,...,9416.726856,6187.414342,2509.585658,40.559522,2024-01-03,2024,1,1,Wednesday,17
9,2024-01-03 14:51:00,Msi Thin 15 B13UCX FHD | Intel Core i7-13620H ...,MSI,Thin,Intel core i7-13620H,RTX 2050,16 GB,4,1715.752230,2425.0,...,10174.088001,6863.008921,2836.991079,41.337424,2024-01-03,2024,1,1,Wednesday,14


In [41]:
# Save the updated datasets
df.to_csv('megapc_products_updated.csv', index=False)
sales_df.to_csv('sales_data.csv', index=False)

print("Files saved:")
print("- megapc_products_updated.csv (products with buying price, selling price, TVA, stock, profit margin)")
print("- sales_data.csv (sales trending data with profit tracking)")

Files saved:
- megapc_products_updated.csv (products with buying price, selling price, TVA, stock, profit margin)
- sales_data.csv (sales trending data with profit tracking)


In [42]:
# Comprehensive sales analysis with profit insights
import matplotlib.pyplot as plt

print("=" * 80)
print("SALES & PROFIT ANALYSIS")
print("=" * 80)

# 1. Daily trends
daily_analysis = sales_df.groupby('date').agg({
    'total_amount': 'sum',
    'total_cost': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum'
}).reset_index()
daily_analysis['profit_margin'] = (daily_analysis['total_profit'] / daily_analysis['total_cost'] * 100)

print("\n📊 DAILY STATISTICS:")
print(f"Average daily revenue: {daily_analysis['total_amount'].mean():,.2f} DT")
print(f"Average daily profit: {daily_analysis['total_profit'].mean():,.2f} DT")
print(f"Best day revenue: {daily_analysis['total_amount'].max():,.2f} DT")
print(f"Best day profit: {daily_analysis['total_profit'].max():,.2f} DT")

# 2. Monthly trends
monthly_analysis = sales_df.groupby(['year', 'month']).agg({
    'total_amount': 'sum',
    'total_cost': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum'
}).reset_index()
monthly_analysis['period'] = monthly_analysis['year'].astype(str) + '-' + monthly_analysis['month'].astype(str).str.zfill(2)
monthly_analysis['profit_margin'] = (monthly_analysis['total_profit'] / monthly_analysis['total_cost'] * 100)

print("\n📅 MONTHLY STATISTICS:")
print(monthly_analysis[['period', 'total_amount', 'total_profit', 'profit_margin', 'quantity']])

# 3. Weekly trends
weekly_analysis = sales_df.groupby(['year', 'week']).agg({
    'total_amount': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum'
}).reset_index()

# 4. Top products by revenue and profit
top_revenue = sales_df.groupby('product_name').agg({
    'quantity': 'sum',
    'total_amount': 'sum',
    'total_profit': 'sum',
    'profit_margin': 'mean'
}).sort_values('total_amount', ascending=False).head(10)

print("\n💰 TOP 10 PRODUCTS BY REVENUE:")
print(top_revenue)

top_profit = sales_df.groupby('product_name').agg({
    'quantity': 'sum',
    'total_amount': 'sum',
    'total_profit': 'sum',
    'profit_margin': 'mean'
}).sort_values('total_profit', ascending=False).head(10)

print("\n💎 TOP 10 PRODUCTS BY PROFIT:")
print(top_profit)

# 5. Brand analysis
brand_analysis = sales_df.groupby('brand').agg({
    'total_amount': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum',
    'profit_margin': 'mean'
}).sort_values('total_profit', ascending=False)

print("\n🏷️  BRAND PERFORMANCE:")
print(brand_analysis)

# 6. GPU performance analysis
gpu_analysis = sales_df.groupby('gpu').agg({
    'total_amount': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum',
    'profit_margin': 'mean'
}).sort_values('total_profit', ascending=False)

print("\n🎮 GPU PERFORMANCE:")
print(gpu_analysis)

# 7. Day of week analysis
dow_analysis = sales_df.groupby('day_of_week').agg({
    'total_amount': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum'
}).reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

print("\n📆 SALES BY DAY OF WEEK:")
print(dow_analysis)

# 8. Hour analysis
hour_analysis = sales_df.groupby('hour').agg({
    'total_amount': 'sum',
    'total_profit': 'sum',
    'quantity': 'sum'
}).reset_index()

print("\n⏰ SALES BY HOUR:")
print(hour_analysis)

print("\n" + "=" * 80)
print(f"TOTAL SUMMARY:")
print(f"Total Revenue: {sales_df['total_amount'].sum():,.2f} DT")
print(f"Total Cost: {sales_df['total_cost'].sum():,.2f} DT")
print(f"Total Profit: {sales_df['total_profit'].sum():,.2f} DT")
print(f"Overall Profit Margin: {(sales_df['total_profit'].sum() / sales_df['total_cost'].sum() * 100):.2f}%")
print("=" * 80)

SALES & PROFIT ANALYSIS

📊 DAILY STATISTICS:
Average daily revenue: 41,446.33 DT
Average daily profit: 11,849.63 DT
Best day revenue: 112,892.77 DT
Best day profit: 33,766.28 DT

📅 MONTHLY STATISTICS:
    period  total_amount   total_profit  profit_margin  quantity
0  2024-01  1.278206e+06  368120.166541      43.410610       448
1  2024-02  9.912058e+05  285002.175886      43.244930       344
2  2024-03  1.136595e+06  315869.672136      41.299475       406
3  2024-04  1.232764e+06  358821.111878      43.745181       434
4  2024-05  1.376595e+06  396038.752477      43.326001       486
5  2024-06  1.160400e+06  332017.272320      42.810572       408
6  2024-07  1.202326e+06  339054.893196      42.257197       418
7  2024-08  1.304815e+06  383202.951780      44.475580       459
8  2024-09  1.307201e+06  371040.848830      42.382138       463
9  2024-10  9.464358e+05  263526.440397      41.515031       332

💰 TOP 10 PRODUCTS BY REVENUE:
                                                    q